# FILE ORGANIZER

___

## 1. Organizzazione dei file

In questo notebook si documenterà la creazione di uno script per organizzare i file contenuti nella cartella "files".
Durante l'esecuzione lo script mostrerà log tipo:
```
bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B
```

I file verranno organizzati in sottocartelle della cartella "files" con una struttura tipo:
```
- files
    - audio
        - song1.mp3
        - song2.mp3
    - docs
        - ciao.txt
        - pippo.odt
    - images
        - bw.png
        - daffodil.jpg
        - eclipse.png
        - trump.jpeg
    - recap.csv
```


Il file recap.csv contine i log creati dallo script.

___

In [1]:
# Librerie necessarie
import os
import magic
import re

In [2]:
# Ottengo il percorso della cartella contenente i file da organizzare
files_directory = "files"

files_path = os.path.join(os.getcwd(), files_directory)

# Controllo se sono presenti le sottocartelle. Se non sono presenti le creo.
sub_directories = ['audio', 'docs', 'images']

for dir in sub_directories:
    if not os.path.isdir(os.path.join(files_path, dir)):
        os.mkdir(os.path.join(files_path, dir))

# Funzione generatore che permette di iterare tutti i file nella cartella file
def get_files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path,file)) and file != 'recap.csv':
            yield file

# Funzione per ottenere il tipo di file
def get_type(file):
    f = magic.Magic(mime=True)
    file_type = f.from_file(os.path.join(files_path, file))
    if bool(re.search(r'audio', file_type)):
        return 'audio'
    elif bool(re.search(r'text', file_type)):
        return 'doc'
    elif bool(re.search(r'image', file_type)):
        return 'image'
    else:
        print(f"{file}: Tipo di file non supportato!")

# Funzione per ottenere la dimensione di un file
def get_size(file):
    return os.path.getsize(os.path.join(files_path, file))

# Funzione per ottenere il nome del file senza estensione
def get_name(file):
    file = os.path.splitext(file)[0]
    return file.split('/')[-1]

for file in get_files(files_path):
    print(f"{get_name(file)} type:{get_type(file)} size:{get_size(file)}B")

song1 type:audio size:1087849B
ciao type:doc size:12B
song2 type:audio size:764176B
daffodil type:image size:24657B
trump type:image size:10195B
bw type:image size:94926B
pippo type:doc size:8299B
eclipse type:image size:64243B


In [3]:
# Per ogni file stampare a schermo nome, tipo e dimensioni. Si sposta il file nella cartella corretta. Infine si salva il log dell'operazione nel file "recap.csv".

## 2. Conversione dello script da notebook a eseguibile

## 3. Utilizzo di NumPy e PIL per mostrare informazionni sulle immagini